В рамках дополнительного проекта попробуем освоить API, позволяющий получать данные о музейных коллекциях.

In [1]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Предположим, ваша задача – найти все произведения, связанные с подсолнухами. Выполните поиск по слову "sunflowers". Внимательно посмотрите на формат ответа и сохраните id полученных объектов в список sunflower_ids.

В качестве ответа выберите id объектов (3), которые присутствуют в полученном списке: [20149, 436524, 16822570, 2032, 437329]

In [6]:
r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?q=sunflowers')

In [11]:
res = r.json()
sunflower_ids = res['objectIDs']

In [13]:
asw = [20149, 436524, 16822570, 2032, 437329]

In [14]:
list(set(sunflower_ids).intersection(set(asw)))

[2032, 437329, 436524]

Следущий этап – собрать информацию об этих объектах. Выполните запрос для объекта 437980.

In [16]:
r_info_by_id = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/437980')

In [19]:
info_by_id = r_info_by_id.json()

Теперь напишите цикл, с помощью которого будет собрана информация об объектах, которые были получены на 2 шаге. Результаты запишите в датафрейм, а именно – следующие параметры:

objectID – id объекта
title – название
artistDisplayName – автор
department – департамент
objectBeginDate – дата (начало)
objectEndDate – дата (конец)
period – название периода
objectName – название/категория объекта
culture – культура
Сколько объектов относятся к японской культуре? (culture –Japan)

In [43]:
sun_req_urls = ['https://collectionapi.metmuseum.org/public/collection/v1/objects/{}'.format(id) for id in sunflower_ids]

In [44]:
info_about_sunflowers_obj = [requests.get(url).json() for url in sun_req_urls]

In [30]:
sunflowers_df = pd.DataFrame(info_about_sunflowers_obj)
sunflowers_df = sunflowers_df[['objectID',
         'title',
         'artistDisplayName',
         'department',
         'objectBeginDate',
         'objectEndDate',
         'period',
         'objectName',
         'culture']]
sunflowers_df.query('culture == "Japan"')

,objectID,title,artistDisplayName,department,objectBeginDate,objectEndDate,period,objectName,culture
29,57922.0,Incense box,,Asian Art,1700.0,1799.0,Edo period (1615–1868),Incense box,Japan


Теперь попробуем построить более сложный запрос с помощью секции "Search"! На этот раз параметры необходимо передать через "&". Выполните поисковый запрос для департамента "Asian art" (departmentId = 6) с тегом "cat" и запишите id объектов в cat_ids. Используйте цикл из шага 4, чтобы получить данные о первой 1000 объектов из списка.

В качестве ответа укажите, в каком периоде (period) было выполнено наибольшее количество работ, представленных в музее?

In [36]:
cats_req = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?departmentId=6&q=cat')
cats_ids = cats_req.json()['objectIDs']

In [41]:
cats_ids_1000 = [cats_ids[ind] for ind in range(1000)]

In [45]:
cats_req_urls = ['https://collectionapi.metmuseum.org/public/collection/v1/objects/{}'.format(id) for id in cats_ids_1000]

In [46]:
info_abouts_cats = [requests.get(url).json() for url in cats_req_urls]

In [47]:
cats_df = pd.DataFrame(info_abouts_cats)

In [48]:
cats_df \
    .groupbyoupbyoupby('period', as_index=False) \
    .agg({'objectID':'count'}) \
    .rename(columns={'objectID':'count'}) \
    .sort_values('count', ascending=False) \
    .head()

,period,count
16,Edo period (1615–1868),442
0,,99
75,Qing dynasty (1644–1911),58
42,Ming dynasty (1368–1644),41
63,Muromachi period (1392–1573),30
